In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

def wait_and_click(driver, by, locator, timeout=10):
    WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((by, locator))
    ).click()

def wait_and_send_keys(driver, by, locator, keys, timeout=10):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((by, locator))
    ).send_keys(keys)

# # Carregar a tabela
tabela = pd.read_excel(r"Caminho para sua tabela")

# Configuração do WebDriver
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

# Abrir a página desejada
navegador.get(r'https://polimed.com.br/autenticadorOrizon/loginAutenticador')
navegador.maximize_window()

# Login
wait_and_send_keys(navegador, By.XPATH, '//*[@id="login"]', 'SEU LOGIN' + Keys.ENTER)
wait_and_send_keys(navegador, By.XPATH, '//*[@id="senha"]', 'SUA SENHA' + Keys.ENTER)
wait_and_click(navegador, By.XPATH, '//*[@id="formLogin"]/button')

# Solicitar senha
wait_and_click(navegador, By.XPATH, '//*[@id="solicitacao_senha_externa"]')

#Loop
for linha in tabela.index:
        # Trocar para o iframe
    iframe_1 = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'conteudoIframe'))
    )
    navegador.switch_to.frame(iframe_1)
    iframe = WebDriverWait(navegador, 10).until(
        EC.presence_of_element_located((By.ID, 'frameOOL'))
    )
    navegador.switch_to.frame(iframe)
    
    # Operadora(Escolher o tipo de operadora, BRADESCO SAÚDE ou BRADESCO SAÚDE OPERADORA DE PLANOS )
    #wait_and_send_keys(navegador, By.XPATH, '//*[@id="select_Ems"]', 'BRADESCO SAÚDE-005711')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="select_Ems"]', 'BRADESCO SAÚDE OPERADORA DE PLANOS-421715')

    # Dados do Beneficiário
    wait_and_click(navegador, By.XPATH, '//*[@id="radio_carteira"]')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="numeroCarteiraBeneficiario"]',str(tabela.loc[linha, "NUMERO DO CARTAO"]))

    wait_and_click(navegador, By.XPATH, '//*[@id="atendimentoRN"]')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="atendimentoRN"]', 'Não')

    # Solicitante
    wait_and_click(navegador, By.XPATH, '//*[@id="tipoContratadoSolicitante"]')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="tipoContratadoSolicitante"]', 'Conselho Profissional' + Keys.ENTER)

    wait_and_click(navegador, By.XPATH, '//*[@id="siglaConselhoContratadoSolicitante"]')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="siglaConselhoContratadoSolicitante"]', 'Conselho Regional de Medicina')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="numeroConselho"]', str(tabela.loc[linha, "CRM"]))
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="ufConselhoContratadoSolicitante"]', tabela.loc[linha, "ESTADO CBOS"] + Keys.ENTER)
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="nomeProfissionalSolicitante"]', tabela.loc[linha, "NOME DO MÉDICO"] + Keys.ENTER)
    wait_and_click(navegador, By.XPATH, '//*[@id="botaoReplicaDados"]')
    wait_and_send_keys(navegador, By.ID, 'CBOSprofissionalSolicitante', str(tabela.loc[linha, "CBOS"]) + Keys.ENTER)

    # Solicitação
    wait_and_click(navegador, By.XPATH, '//*[@id="radioCaraterSolicitacaoEletiva"]')

    # Contratado
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="matriculaPrestadorContratadoExecutante"]', 'Sua Matricula' + Keys.ENTER)

    # Info.ad
    wait_and_send_keys(navegador, By.ID, 'indicacaoClinica', str(tabela.loc[linha, "INDICAÇÃO CLÍNICA"]) + Keys.ENTER)

    # Procedimento
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="codigoProcedimento"]', 'Codigo do seu procedimento' + Keys.ENTER)
    wait_and_click(navegador, By.XPATH, '//*[@id="AdicionarProcedimento"]')
    wait_and_send_keys(navegador, By.XPATH, '//*[@id="codigoProcedimento"]', 'Codigo do seu procedimento' + Keys.ENTER)
    wait_and_click(navegador, By.ID, 'tdUploadArquivo')

    time.sleep(15)
    
    # Executar envio
    navegador.find_element(By.ID, 'enviar').click()

    # Solicitar nova senha
    navegador.refresh()

#Fechar o navegador
navegador.find_element(By.ID, 'sair').click()